In [ ]:
import json
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
from shapely.ops import split
import numpy as np
from scipy.optimize import linprog
import random
import graphviz

In [ ]:
with open('trial_model.json') as in_file:
    asterism = json.load(in_file)
input_point = np.zeros((60))

In [ ]:
def random_color():
    return "#"+''.join([random.choice('0123456789ABCDEF') for _ in range(6)])

In [ ]:
nodes = asterism['arena']
root_node = nodes[0]
if 'input_bounds_opt' in asterism:
    bounds = asterism['input_bounds_opt']['data']
    bounds = np.reshape(np.array(bounds['data']), bounds['dim'])
    bounds = np.transpose(bounds[:,:])[-2:]
print(f'{bounds=}')
parents = asterism['parents']
node_colors = [random_color() for _ in enumerate(parents)]

cons_coeffs = []
cons_rhs = []
for node in nodes:
    con = node['star']['constraints']
    if con is None:
        cons_coeffs.append(None)
        cons_rhs.append(None)
        continue

    coeffs = con['coeffs']
    coeffs = np.reshape(coeffs['data'], coeffs['dim'])
    rhs = con['rhs']
    rhs = np.reshape(rhs['data'], rhs['dim'])

    rhs_offset = np.dot(coeffs[:, :input_point.shape[0]], input_point)
    coeffs = coeffs[:, input_point.shape[0]:]
    rhs = rhs - rhs_offset

    cons_coeffs.append(coeffs)
    cons_rhs.append(rhs)

def check_ineq(i, point):
    ineq_test = np.matmul(cons_coeffs[i], point) <= cons_rhs[i]
    return np.all(ineq_test)    

In [ ]:
boxes = []
points = []

bounds_box = Polygon([(bounds[0,0], bounds[1,0]), (bounds[0,1], bounds[1,0]), (bounds[0,1], bounds[1,1]),
                  (bounds[0,0], bounds[1,1]), (bounds[0,0], bounds[1,0])])
boxes.append(bounds_box)

for i, parent in enumerate(parents):
    if parent is None:
        continue

    if cons_coeffs[i] is None:
        line = None
    elif np.all(cons_coeffs[i][-1] == 0.) and cons_rhs[i][-1] == 0.:
        line = None
    elif np.all(cons_coeffs[i][-1] == 0.):
        print('Reached contradictory polygon')
        line = None
    elif isinstance(boxes[parent], Point):
        line = None
    elif cons_coeffs[i][-1][1] == 0.:
        x = cons_rhs[i][-1] / cons_coeffs[i][-1][0]
        line = LineString([(x, bounds[1, 0]), (x, bounds[1, 1])])
    else:
        slope = -cons_coeffs[i][-1][0] / cons_coeffs[i][-1][1]
        intercept = cons_rhs[i][-1] / cons_coeffs[i][-1][1]
        left = (bounds[0, 0], slope * bounds[0, 0] + intercept)
        right = (bounds[0, 1], slope * bounds[0, 1] + intercept)
        line = LineString([left, right])

    if line:
        parent_poly = boxes[parent]
        child_polys = split(parent_poly, line)
        geom_added = False
        for geom in child_polys.geoms:
            if check_ineq(i, np.array([geom.centroid.x, geom.centroid.y])):
                boxes.append(geom)
                geom_added = True
                break
        if not geom_added:
            res = linprog([0, 1],
                A_ub=cons_coeffs[i],
                b_ub=cons_rhs[i])
            assert res.success
            boxes.append(Point(*res.x))
    else:
        boxes.append(boxes[parent])


In [ ]:
for i, _ in enumerate(parents):
    print(i)
    i = i + 1
    fig, axs = plt.subplots()
    for geom, color in zip(boxes[:i], node_colors[:i]):
        if isinstance(geom, Point):
            c = np.asarray(geom.coords)
            axs.plot(c, marker='o', color=color)
            continue

        xs, ys = geom.exterior.xy
        axs.fill(xs, ys, color=color, ec='none')
    plt.show()

In [ ]:
g = graphviz.Graph('graph')

for i, c in enumerate(node_colors):
    g.node(str(i), style='filled', fillcolor=c)

for i, p in enumerate(parents):
    if p is None:
        continue

    g.edge(str(p), str(i))

g.save()